In [1]:
import numpy as np
import faiss  # FAISS（フェイス）のインポート

# パラメータ設定
d = 64       # ベクトルの次元数
nb = 10000   # データベースのベクトル数
nq = 10      # クエリのベクトル数

np.random.seed(1234)
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.  # わずかなシフトを付与
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

# L2距離を用いるフラットなインデックス作成
index = faiss.IndexFlatL2(d)
index.add(xb)  # インデックスにデータを追加

# インデックス情報をローカルファイルに出力
faiss.write_index(index, "faiss_index.index")

# ローカルファイルからインデックスを読み込み
index_loaded = faiss.read_index("faiss_index.index")

# 読み込んだインデックスで検索（各クエリに対し最も近い4件を探索）
k = 4
D, I = index_loaded.search(xq, k)

print("近傍のインデックス:", I)
print("各ベクトル間の距離:", D)


近傍のインデックス: [[ 794   72  515 1204]
 [ 393  919  424  923]
 [ 688  806  763  185]
 [ 950  230  317  703]
 [ 199  409  411  794]
 [ 383  227 1075  416]
 [ 630  133   32  917]
 [1407  235  779  470]
 [  14   22  581  507]
 [ 637  782  483  680]]
各ベクトル間の距離: [[5.9375467 6.2643566 6.2803383 6.3964715]
 [7.596614  8.063351  8.070611  8.196373 ]
 [7.180752  7.3030562 7.459653  7.502161 ]
 [6.0598154 6.094823  6.1986046 6.550714 ]
 [5.482876  7.1775966 7.246011  7.33661  ]
 [6.25152   6.906644  7.0537624 7.1303234]
 [6.4409313 6.578717  6.6783037 6.6917696]
 [7.0216146 7.0476847 7.33723   7.4447927]
 [7.4367833 7.784588  7.9513555 7.968199 ]
 [5.997012  6.421344  6.9501457 7.0086737]]


In [2]:
import os
import logging
import numpy as np
import faiss  # FAISS（フェイス）のインポート

# ロガーの設定
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class FaissManager:
    def __init__(self, dim, index_file=None, recreate=True):
        """
        Args:
            dim (int): 埋め込みベクトルの次元数
            index_file (str, optional): インデックス保存用ファイルパス
            recreate (bool): Trueの場合、既存のファイルがあっても新規作成します。
        """
        self.dim = dim
        self.index_file = index_file
        self.file_paths = []  # 各ベクトルに対応する画像ファイルパスのリスト

        if index_file is not None and os.path.exists(index_file) and not recreate:
            self.index = faiss.read_index(index_file)
            logger.info("FAISSインデックスを読み込みました。ファイル: %s", index_file)
        else:
            # 正規化済みの特徴量を使い、内積検索（IndexFlatIP）を利用します。
            self.index = faiss.IndexFlatIP(dim)
            logger.info("新しいFAISSインデックスを作成しました。")

    def add_embeddings(self, embeddings, file_paths):
        """
        インデックスに埋め込みベクトルを追加するメソッドです。
        Args:
            embeddings: 埋め込みベクトルのリストまたはNumPy配列
            file_paths: 各ベクトルに対応する画像ファイルパスのリスト
        """
        # 埋め込みをfloat32型の連続メモリのNumPy配列に変換
        embeddings = np.ascontiguousarray(np.array(embeddings, dtype=np.float32))
        if embeddings.ndim != 2 or embeddings.shape[1] != self.dim:
            raise ValueError(f"埋め込みの次元が不正です。期待値: (N, {self.dim})")
        self.index.add(embeddings)
        self.file_paths.extend(file_paths)
        logger.info("%d 件の埋め込みをインデックスに追加しました。", len(file_paths))

    def search(self, query_vector, k):
        """
        クエリベクトルに対して近傍検索を行います。
        Args:
            query_vector: 検索対象のクエリベクトル（1次元の場合は自動的に変換）
            k (int): 返す近傍件数
        Returns:
            distances, indices: 各クエリベクトルに対する距離とインデックスの結果
        """
        query_vector = np.ascontiguousarray(np.array(query_vector, dtype=np.float32))
        if query_vector.ndim == 1:
            query_vector = query_vector.reshape(1, -1)
        if query_vector.shape[1] != self.dim:
            raise ValueError(f"次元不一致: インデックスは {self.dim} 次元ですが、クエリは {query_vector.shape[1]} 次元です。")
        distances, indices = self.index.search(query_vector, k)
        return distances, indices

    def save(self, index_file=None):
        """
        FAISSインデックスをローカルファイルに保存します。
        Args:
            index_file (str, optional): 保存するファイルパス。Noneの場合、初期化時のindex_fileを使用。
        """
        target_file = index_file if index_file is not None else self.index_file
        if target_file is None:
            raise ValueError("保存するためのファイルパスが指定されていません。")
        faiss.write_index(self.index, target_file)
        logger.info("FAISSインデックスをファイルに保存しました: %s", target_file)

# 動作確認の例
if __name__ == "__main__":
    # パラメータ設定
    d = 64       # 次元数
    nb = 1000    # データベースのベクトル数
    nq = 5       # クエリのベクトル数

    # ランダムなデータを生成（例として）
    np.random.seed(1234)
    xb = np.ascontiguousarray(np.random.random((nb, d)).astype('float32'))
    xq = np.ascontiguousarray(np.random.random((nq, d)).astype('float32'))

    # FAISSマネージャーのインスタンスを作成し、埋め込みを追加
    manager = FaissManager(dim=d, index_file="faiss_index.index", recreate=True)
    # ここではファイルパスはダミーの文字列を利用
    manager.add_embeddings(xb, [f"image_{i}.jpg" for i in range(nb)])

    # インデックスを保存
    manager.save()

    # 検索実行（各クエリに対して上位4件を取得）
    distances, indices = manager.search(xq, k=4)
    print("検索結果のインデックス:", indices)
    print("検索結果の距離:", distances)


INFO:__main__:新しいFAISSインデックスを作成しました。
INFO:__main__:1000 件の埋め込みをインデックスに追加しました。
INFO:__main__:FAISSインデックスをファイルに保存しました: faiss_index.index


検索結果のインデックス: [[830  96 973  30]
 [830 890 916  96]
 [830 148 473 973]
 [890 900 596 600]
 [890 830 710 805]]
検索結果の距離: [[20.27583  20.191795 19.769772 19.723906]
 [17.27805  17.063442 16.98305  16.83917 ]
 [19.777376 18.932695 18.58435  18.551655]
 [21.178482 20.678907 20.246483 20.19095 ]
 [23.13449  21.879822 21.373438 21.22133 ]]
